In [1]:
!pip install kafka-python
!pip install redis

     |████████████████████████████████| 266kB 511kB/s eta 0:00:01    |██████▏                         | 51kB 995kB/s eta 0:00:01
     |████████████████████████████████| 71kB 475kB/s eta 0:00:01


In [2]:
import os
import requests
import json
import redis
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from datetime import datetime

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[2] pyspark-shell'
conf = SparkConf().set("spark.jars","/home/jovyan/work/spark-streaming-kafka-0-8-assembly_2.11-2.4.4.jar")
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,60) # 60 is the batch interval :)
debug = True
saveToLocalDisk = True
spark = SparkSession(sc)

In [4]:
def SetVal(x):
    r = redis.StrictRedis(host = 'redis-store', port = 6379)
    r.set('LatestNews',"{\"tasks\":["+str(x)+"]}")
        
def SaveToNewsFile(rdd):
    if not rdd.isEmpty():
        rdd.toDF() \
        .write.save("/home/jovyan/work/NewsData.csv",
                    maxRecordsPerFile=1000, format="csv", mode="append") 
    
def SaveToFile(rdd):
    if not rdd.isEmpty():
        rdd.toDF( [ "Ticker", "Timestamp", "Price" ] ) \
        .write.save("/home/jovyan/work/FinanceData.csv",
                    maxRecordsPerFile=1000, format="csv", mode="append") 
        
def ConvertToTuple(a):
    if not (type(a) is list): 
        return (a['Timestamp'],a['Title'])
    else:
        return None

In [ ]:
kafkaStream = KafkaUtils.createStream(
    ssc=ssc, 
    zkQuorum='zookeeper:2181', 
    groupId='test-consumer-group',#, #Consumer Group 
    topics={'finance_ticker':1,'news_feed':2})

olines = kafkaStream.map(lambda x: x[1]).map(lambda a: json.loads(a))
if(debug):
    olines.pprint()
if(saveToLocalDisk):
    olines.filter(lambda a: ('Type' in a) & (a['Type']=="Finance")) \
    .map(lambda a: (a['Ticker'],a['Timestamp'],a['Price'])) \
    .foreachRDD(SaveToFile)
    
    olines.filter(lambda a: ('Type' in a) & (a['Type']=="News")) \
    .map(lambda a: (a['Timestamp'],a['Title'])) \
    .foreachRDD(SaveToNewsFile)
    
lines = olines.filter(lambda a: ('Type' in a) & (a['Type']=="Finance")) \
        .map(lambda a: str(a['Price'])) \
        .reduce(lambda v,agg: agg+", "+v)

if(debug):
    lines.pprint()

lines = olines.filter(lambda a: ('Type' in a) & (a['Type']=="News")) \
        .map(lambda a: json.dumps(a)).reduce(lambda v,agg: agg+", "+v) \
        .map(lambda x: SetVal(x))
    
if(debug):
    lines.pprint()
    
ssc.start()  
print("Going to wait termination")
ssc.awaitTermination()

Going to wait termination
-------------------------------------------
Time: 2019-12-09 05:23:00
-------------------------------------------
{'Type': 'News', 'Timestamp': 1575868951.028209, 'Title': 'Jobs of the future are clustering in a handful of U.S. cities, study finds', 'Summary': 'A new analysis of where "innovation" jobs are being created in the United States paints a stark portrait of a divided economy where the industries seen as key to future growth cluster in a narrowing set of places.<div class="feedflare">\n<a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=_jMBnENOkTs:pzAwlDZxLzA:yIl2AUoC8zA"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?d=yIl2AUoC8zA" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/businessNews?a=_jMBnENOkTs:pzAwlDZxLzA:F7zBnMyn0Lo"><img src="http://feeds.feedburner.com/~ff/reuters/businessNews?i=_jMBnENOkTs:pzAwlDZxLzA:F7zBnMyn0Lo" border="0"></img></a> <a href="http://feeds.reuters.com/~ff/reuters/business

-------------------------------------------
Time: 2019-12-09 05:23:00
-------------------------------------------
270.71, 151.75

-------------------------------------------
Time: 2019-12-09 05:23:00
-------------------------------------------
None

-------------------------------------------
Time: 2019-12-09 05:24:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:24:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:24:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:25:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:25:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09 05:25:00
-------------------------------------------

-------------------------------------------
Time: 2019-12-09

In [ ]:
lines = lines.map(lambda a: requests.post(
        "http://serving:8501/v1/models/half_plus_two:predict",
        "{\"instances\": ["+a+"]}").text)
if(debug):
    lines.pprint()

lines = lines.flatMap(lambda x: json.loads(x)["predictions"])
if(debug):
    lines.pprint()

lines = lines.map(lambda x: SetVal(x))
if(debug):
    lines.pprint()